### Simple CNN - MNIST Classification

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
USE_CUDA = torch.cuda.is_available()
device = ("cuda" if USE_CUDA else "cpu")
print(device)

if device == "cude":
  torch.cuda.manual_seed(777)

else:
  torch.manual_seed(1)

cuda


In [4]:
mnist_train = datasets.MNIST("/datasets/MNIST", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST("/datasets/MNIST", train=False, transform=transforms.ToTensor(), download=True)

train_dataloader = DataLoader(mnist_train, batch_size=128, shuffle=True, drop_last=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 349748986.27it/s]

Extracting /datasets/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /datasets/MNIST/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 111851979.52it/s]


Extracting /datasets/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /datasets/MNIST/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 163926412.02it/s]

Extracting /datasets/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /datasets/MNIST/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 8516105.84it/s]


Extracting /datasets/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /datasets/MNIST/MNIST/raw



In [5]:
# Define CNN

class SimpleCNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer3 = nn.Linear(7 * 7 * 64, 10, bias=True)


  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = torch.flatten(out, 1)
    return self.layer3(out)

model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

total_epochs = 15


for epoch in range(total_epochs + 1):

  avg_cost = 0
  total_batch = len(train_dataloader)

  for batch_idx, sample in enumerate(train_dataloader):

    x_batch, y_batch = sample
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    y_pred = model(x_batch)

    cost = criterion(y_pred, y_batch)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print(f"Epoch : {epoch}/{total_epochs} : Cost = {avg_cost.item():.4f}")

Epoch : 0/15 : Cost = 0.2335
Epoch : 1/15 : Cost = 0.0632
Epoch : 2/15 : Cost = 0.0460
Epoch : 3/15 : Cost = 0.0383
Epoch : 4/15 : Cost = 0.0315
Epoch : 5/15 : Cost = 0.0282
Epoch : 6/15 : Cost = 0.0231
Epoch : 7/15 : Cost = 0.0205
Epoch : 8/15 : Cost = 0.0187
Epoch : 9/15 : Cost = 0.0145
Epoch : 10/15 : Cost = 0.0124
Epoch : 11/15 : Cost = 0.0115
Epoch : 12/15 : Cost = 0.0097
Epoch : 13/15 : Cost = 0.0096
Epoch : 14/15 : Cost = 0.0081
Epoch : 15/15 : Cost = 0.0064


In [6]:
test_dataloader = DataLoader(mnist_test, batch_size=128, shuffle=True, drop_last=True)

with torch.no_grad():

  total_sample = 0
  accuracy = 0

  for batch_idx, sample in enumerate(test_dataloader):

    x_batch, y_batch = sample
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    y_pred = model(x_batch)
    accuracy += ((y_pred.argmax(dim=1) == y_batch).count_nonzero())
    total_sample += len(y_batch)

  print(f"Accuracy : {(accuracy / total_sample) * 100:.2f}%")

Accuracy : 98.98%


### Deep CNN - MNIST Classification

In [7]:
# Define CNN

class DeepCNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )

    self.layer4 = nn.Sequential(
        nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )

    self.layer5 = nn.Linear(3 * 3 * 64, 10, bias=True)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = torch.flatten(out, 1)
    return self.layer5(out)


model = DeepCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

total_epochs = 15

for epoch in range(total_epochs + 1):

  avg_cost = 0
  total_batch = len(train_dataloader)

  for batch_idx, sample in enumerate(train_dataloader):

    x_batch, y_batch = sample
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    y_pred = model(x_batch)

    cost = criterion(y_pred, y_batch)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print(f"Epoch : {epoch}/{total_epochs} : Cost = {avg_cost.item():.4f}")

Epoch : 0/15 : Cost = 0.2572
Epoch : 1/15 : Cost = 0.0584
Epoch : 2/15 : Cost = 0.0407
Epoch : 3/15 : Cost = 0.0283
Epoch : 4/15 : Cost = 0.0229
Epoch : 5/15 : Cost = 0.0200
Epoch : 6/15 : Cost = 0.0167
Epoch : 7/15 : Cost = 0.0142
Epoch : 8/15 : Cost = 0.0108
Epoch : 9/15 : Cost = 0.0107
Epoch : 10/15 : Cost = 0.0088
Epoch : 11/15 : Cost = 0.0095
Epoch : 12/15 : Cost = 0.0089
Epoch : 13/15 : Cost = 0.0077
Epoch : 14/15 : Cost = 0.0066
Epoch : 15/15 : Cost = 0.0064


In [8]:
test_dataloader = DataLoader(mnist_test, batch_size=128, shuffle=True, drop_last=True)

with torch.no_grad():

  total_sample = 0
  accuracy = 0

  for batch_idx, sample in enumerate(test_dataloader):

    x_batch, y_batch = sample
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    y_pred = model(x_batch)
    accuracy += ((y_pred.argmax(dim=1) == y_batch).count_nonzero())
    total_sample += len(y_batch)

  print(f"Accuracy : {(accuracy / total_sample) * 100:.2f}%")

Accuracy : 99.07%
